## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Rikitea,-23.1203,-134.9692,74.46,83,100,12.82,moderate rain
1,1,Barentu,15.1139,37.5928,76.53,45,97,7.05,overcast clouds
2,2,Saint George,37.1041,-113.5841,59.63,43,1,1.99,clear sky
3,3,Vardo,70.3705,31.1107,35.24,57,100,23.40,overcast clouds
4,4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,59,0,7.20,clear sky


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Rikitea,-23.1203,-134.9692,74.46,83,100,12.82,moderate rain
1,1,Barentu,15.1139,37.5928,76.53,45,97,7.05,overcast clouds
4,4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,59,0,7.20,clear sky
8,8,Butaritari,3.0707,172.7902,82.72,73,15,10.29,few clouds
16,16,Pringsewu,-5.3582,104.9744,84.11,70,100,3.11,overcast clouds
19,19,Cabedelo,-6.9811,-34.8339,75.87,75,4,11.95,clear sky
21,21,Saint-Philippe,-21.3585,55.7679,71.15,74,17,10.02,few clouds
25,25,Hobart,-42.8794,147.3294,71.22,51,20,3.00,few clouds
27,27,Hilo,19.7297,-155.0900,80.33,74,90,8.05,overcast clouds
32,32,Airai,-8.9266,125.4092,79.68,35,96,2.82,overcast clouds


In [60]:
# 4a. Determine if there are any empty rows.
empty_string = (hotel_df["City"]=="")
preferred_cities_df["City"].loc[empty_string].count()

0

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace('', np.nan)
preferred_cities_df = preferred_cities_df.dropna()

In [55]:
preferred_cities_df.count()

City_ID                270
City                   270
Lat                    270
Lng                    270
Max Temp               270
Humidity               270
Cloudiness             270
Wind Speed             270
Weather Description    270
dtype: int64

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Lat", "Lng", "Max Temp", "Weather Description", ]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Weather Description,Hotel Name
0,Rikitea,-23.1203,-134.9692,74.46,moderate rain,
1,Barentu,15.1139,37.5928,76.53,overcast clouds,
4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,clear sky,
8,Butaritari,3.0707,172.7902,82.72,few clouds,
16,Pringsewu,-5.3582,104.9744,84.11,overcast clouds,
19,Cabedelo,-6.9811,-34.8339,75.87,clear sky,
21,Saint-Philippe,-21.3585,55.7679,71.15,few clouds,
25,Hobart,-42.8794,147.3294,71.22,few clouds,
27,Hilo,19.7297,-155.0900,80.33,overcast clouds,
32,Airai,-8.9266,125.4092,79.68,overcast clouds,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
##params = {"&radius=": 5000, "&type=": "lodging",  "&key=": g_key}
radius_param = 5000
type_param = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the URL for the Google Directions API
    hotel_url = f"{base_url}location={lat}%2C{lng}&radius=5000&type={type_param}&key={g_key}"
     ##"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522%2C151.1957362&radius=1500&type=restaurant&key=YOUR_API_KEY"  
        
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(hotel_url).json()
     
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [61]:
hotel_df

,City,Lat,Lng,Max Temp,Weather Description,Hotel Name
0,Rikitea,-23.1203,-134.9692,74.46,moderate rain,People ThankYou
1,Barentu,15.1139,37.5928,76.53,overcast clouds,
4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,clear sky,Hotel Tala Puera
8,Butaritari,3.0707,172.7902,82.72,few clouds,Isles Sunset Lodge
16,Pringsewu,-5.3582,104.9744,84.11,overcast clouds,Urban Style by Front One
...,...,...,...,...,...,...
671,Puerto Colombia,10.9878,-74.9547,84.29,overcast clouds,Hotel El Emigrante
672,Lakatoro,-16.0999,167.4164,83.10,few clouds,Lakatoro Palm Lodge
674,Poya,-21.3500,165.1500,79.90,overcast clouds,
676,Kerema,-7.9631,145.7785,85.30,overcast clouds,Frena Lodge


In [62]:
#hotel_df.count() # is counting empty strings

In [63]:
empty_string = (hotel_df["Hotel Name"]=="")
hotel_df["Hotel Name"].loc[empty_string].count()

24

In [71]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Weather Description,Hotel Name
0,Rikitea,-23.1203,-134.9692,74.46,moderate rain,People ThankYou
1,Barentu,15.1139,37.5928,76.53,overcast clouds,NaN
4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,clear sky,Hotel Tala Puera
8,Butaritari,3.0707,172.7902,82.72,few clouds,Isles Sunset Lodge
16,Pringsewu,-5.3582,104.9744,84.11,overcast clouds,Urban Style by Front One
...,...,...,...,...,...,...
671,Puerto Colombia,10.9878,-74.9547,84.29,overcast clouds,Hotel El Emigrante
672,Lakatoro,-16.0999,167.4164,83.10,few clouds,Lakatoro Palm Lodge
674,Poya,-21.3500,165.1500,79.90,overcast clouds,NaN
676,Kerema,-7.9631,145.7785,85.30,overcast clouds,Frena Lodge


In [72]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Weather Description,Hotel Name
0,Rikitea,-23.1203,-134.9692,74.46,moderate rain,People ThankYou
4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,clear sky,Hotel Tala Puera
8,Butaritari,3.0707,172.7902,82.72,few clouds,Isles Sunset Lodge
16,Pringsewu,-5.3582,104.9744,84.11,overcast clouds,Urban Style by Front One
19,Cabedelo,-6.9811,-34.8339,75.87,clear sky,Hotel Pigalle
...,...,...,...,...,...,...
668,Saint Pete Beach,27.7253,-82.7412,78.85,clear sky,Grand Plaza Hotel - St. Pete Beach
671,Puerto Colombia,10.9878,-74.9547,84.29,overcast clouds,Hotel El Emigrante
672,Lakatoro,-16.0999,167.4164,83.10,few clouds,Lakatoro Palm Lodge
676,Kerema,-7.9631,145.7785,85.30,overcast clouds,Frena Lodge


In [73]:
empty_string = (hotel_df["Hotel Name"]=="")
hotel_df["Hotel Name"].loc[empty_string].count()

0

In [74]:
hotel_df

,City,Lat,Lng,Max Temp,Weather Description,Hotel Name
0,Rikitea,-23.1203,-134.9692,74.46,moderate rain,People ThankYou
4,General Jose Eduvigis Diaz,-27.1667,-58.4167,73.44,clear sky,Hotel Tala Puera
8,Butaritari,3.0707,172.7902,82.72,few clouds,Isles Sunset Lodge
16,Pringsewu,-5.3582,104.9744,84.11,overcast clouds,Urban Style by Front One
19,Cabedelo,-6.9811,-34.8339,75.87,clear sky,Hotel Pigalle
...,...,...,...,...,...,...
668,Saint Pete Beach,27.7253,-82.7412,78.85,clear sky,Grand Plaza Hotel - St. Pete Beach
671,Puerto Colombia,10.9878,-74.9547,84.29,overcast clouds,Hotel El Emigrante
672,Lakatoro,-16.0999,167.4164,83.10,few clouds,Lakatoro Palm Lodge
676,Kerema,-7.9631,145.7785,85.30,overcast clouds,Frena Lodge


In [75]:
# 8a. Create the output File (CSV)
output_csv_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_csv_file, index_label="City_ID")

In [76]:
# 9. Using the template add city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [77]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))